In [ ]:
from pathlib import Path

import torch
from torch.utils.data import DataLoader

from src.data.dataset import ObjectDetectionDataset
from src.data.entry import read_entries_from_directory, split_entries_train_val_test
from src.data.visualize import plot_entries_original_and_annotated
from src.trainer import Trainer
from src.utils import load_config

print("Pytorch version:", torch.__version__)
print("CUDA enabled:", torch.cuda.is_available())

In [ ]:
DEVICE = torch.device("cuda" if torch.cuda.is_available else "cpu")
DATASET_DIR = Path("C:/Users/robert/Desktop/sem1/NN/datasets/DetectionPatches_512x512_ALL")
CHECKPOINT_DIR = Path("models")
CONFIG_PATH = Path("config.json")
SINGLE_CLASS = True

In [ ]:
config = load_config(CONFIG_PATH)
seed = config["seed"]
torch.manual_seed(seed)
config

In [ ]:
entries = read_entries_from_directory(DATASET_DIR)
train_entries, val_entries, test_entries = split_entries_train_val_test(entries, seed=seed)

In [ ]:
train_dataset = ObjectDetectionDataset(train_entries, single_class=SINGLE_CLASS, config=config)
val_dataset = ObjectDetectionDataset(val_entries, single_class=SINGLE_CLASS, config=config)
test_dataset = ObjectDetectionDataset(test_entries, single_class=SINGLE_CLASS, config=config)

num_positive_entries = train_dataset.num_positive + val_dataset.num_positive + test_dataset.num_positive
num_negative_entries = train_dataset.num_negative + val_dataset.num_negative + test_dataset.num_negative
objects_per_entry = [len(entry.annotations) for entry in entries]
total_objects = sum(objects_per_entry)
max_objects = max(objects_per_entry)

print(f"Total number of entries: {len(entries)}")
print(f"Total number of positive entries: {num_positive_entries} ({num_positive_entries / len(entries) * 100:.2f}%)")
print(f"Total number of negative entries: {num_negative_entries} ({num_negative_entries / len(entries) * 100:.2f}%)")
print(f"Total number of unique objects (cars): {total_objects}")
print(f"Average number of objects per positive entry: {total_objects / num_positive_entries:.2f}")
print(f"Max number of objects in an entry: {max_objects}")
print(train_dataset.describe("TRAIN"))
print(val_dataset.describe("VAL"))
print(test_dataset.describe("TEST"))

In [ ]:
max_annotation_entry = max(entries, key=lambda x: len(x.annotations), default=None)

if max_annotation_entry is not None:
    plot_entries_original_and_annotated([max_annotation_entry], samples_to_display=1)

In [ ]:
plot_entries_original_and_annotated(train_dataset.entries, samples_to_display=1)

In [ ]:
batch_size = config["batch_size"]
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=False, pin_memory=True)
val_dataloader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, pin_memory=True)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, pin_memory=True)

In [ ]:
trainer = Trainer(
    config=config,
    dataloaders=(train_dataloader, val_dataloader, test_dataloader),
    device=DEVICE,
    save_dir=CHECKPOINT_DIR,
    checkpoint_path=None
)

In [ ]:
trainer.fit()

In [ ]:
trainer.evaluate(test_dataloader)